## Hartree's method to calculate He energy and wavefunction


We start writing the Schrodinger equation: $$\large \hat{H}\psi(\bar{r_1},\bar{r_2}) = E\psi(\bar{r_1},\bar{r_2})$$
where $$\large \hat{H} = -\frac{\hbar^2}{2m}\nabla^2_1 -\frac{\hbar^2}{2m}\nabla^2_2 - \frac{Ze^2}{4\pi\epsilon_0 r_1} - \frac{Ze^2}{4\pi\epsilon_0 r_2} + \frac{e^2}{4\pi\epsilon_0 |r_1-r_2|}$$

It is evident that this hamiltonian can be expressed as a sum os three different hamiltonian: $$\large \hat{H} = \hat{H_1} + \hat{H_2} + \hat{H}_{12} $$
where the first(second) term is the sum of the first(second) and third(fourth) term of the general hamiltonian and describes the interaction of the first(second) electron with the nucleus, while the last term is the repulsion potential between the two electrons. Therefore, if we consider $\hat{H_{12}}$ negligible, the energy eigenvalue wuold be the sum of the two electron's energies in the hydrogenoid atom. However, the experimental evidence results in a very different value of the energy, outlining a non-vanishing interaction contribution to the total energy of the atom.

The Hartree Method is a computational approach at solving this hamiltonian.
We assume that hydrogen wavefunctions and energies are all we need to perform our calculations. In the hydrogenoid atom, the eigenfunctions are: $$\large \psi(r,\theta, \phi) = \mathcal{R}_{n,l}(r)Y_{l,m_l}(\theta, \phi) $$ with eigenvalues $$\large E_n = -E_{Ry}\frac{Z^2}{n^2}$$
We write our solution as $$\large \psi(\bar{r_1},\bar{r_2}) = \psi_{\alpha}(\bar{r_1})\psi_{\beta}(\bar{r_2})$$
where $\alpha$ and $\beta$ are the electrons quantum numbers.

We can write the expectation value of the potential energy energy for the first electron as $$\large E_p(\bar{r_1})= -\frac{Ze^2}{4 pi\epsilon_0r_1} + \braket{E} $$
where $\braket{E}$ is the mean potential energy added to the first electron due to interaction with the second electron. We now calculate the expectation value of the last term of the hamiltonian for the second electron $$\large \begin{align*} \braket{\hat{H}_{12}} &= \bra{\psi_{\beta}(\bar{r_2})}\frac{e^2}{4\pi\epsilon_0|\bar{r_1}-\bar{r_2}|}\ket{\psi_{\beta}(\bar{r_2})} \\ &= \int d^3r_2 |\psi_{\beta}(\bar{r_2})|^2\frac{e^2}{4\pi\epsilon_0|\bar{r_1}-\bar{r_2}|}          \end{align*} $$

In [ ]:
# conversion from Hartree to eV
eV = 27.2113966413442 # 1 Hartree = 2 Rydberg, Bohr radius a_0 = 1, electron mass = 1, h/4pi = 1



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse, linalg
from scipy.sparse import linalg as sla


def schrodinger3D(xmin, xmax, Nx, 
                  ymin, ymax, Ny, 
                  zmin, zmax, Nz, 
                  Vfun3D, params, neigs, E0=0.0, findpsi=False):
    """
    This function solves the 3 dimensional Schrodinger equation numerically.
    """
    x = np.linspace(xmin, xmax, Nx)  
    dx = x[1] - x[0]  
    y = np.linspace(ymin, ymax, Ny)
    dy = y[1] - y[0]
    z = np.linspace(zmin, zmax, Nz)
    dz = z[1] - z[0]
 
    V = Vfun3D(x, y, z, params)
 
    # create the 3D Hamiltonian matrix
    Hx = create_hamiltonian(Nx, dx)
    Hy = create_hamiltonian(Ny, dy)
    Hz = create_hamiltonian(Nz, dz)
     
    Ix = sparse.eye(Nx)
    Iy = sparse.eye(Ny)
    Iz = sparse.eye(Nz)
     
    # Combine the 3 individual 1 dimensional Hamiltonians
    # using Kronecker products
    Hxy = sparse.kron(Iy, Hx) + sparse.kron(Hy, Ix)
    Ixy = sparse.kron(Iy, Ix)
    H = sparse.kron(Iz, Hxy) + sparse.kron(Hz, Ixy)
     
    # Convert to lil form and add potential energy function
    H = H.tolil()
    for i in range(Nx * Ny * Nz):
        H[i, i] = H[i, i] + V[i]    
 
    # convert to csc form and solve the eigenvalue problem
    H = H.tocsc()  
    [evl, evt] = sla.eigs(H, k=neigs, sigma=E0)
             
    if findpsi == False:
        return evl
    else: 
        return evl, evt, x, y, z
     
def create_hamiltonian(Nx, dx):
    """
    This function creates a 1 dimensional numerical Hamiltonian.
    """
    H = sparse.eye(Nx, Nx, format='lil') * 2
    for i in range(Nx - 1):
        H[i, i + 1] = -1
        H[i + 1, i] = -1
    H = H / (dx ** 2)  
    return H

In [2]:
def sho_eigenenergies(xmin = -5, xmax = 5, Nx = 50, ymin = -5, ymax = 5, Ny = 50, 
                      zmin = -5, zmax = 5, Nz = 50, params = [1, 1, 1], neigs = 1, E0 = 0):
    """
    This function calculates the quantum simple harmonic oscillator eigenenergies.
    Theoretically, the eigenenergies are given by: E = hw(n + 3/2), n = nx + ny + nz.
    However, as we set h = w = 1, and we scale the energies during the Hamiltonian creation
    by 2, the theoretical eigenenergies are given by: E = 2n + 3.
    """
    def Vfun(X, Y, Z, params):
        Nx = len(X)
        Ny = len(Y)
        Nz = len(Z)
        M = Nx * Ny * Nz
        V = np.zeros(M)
        vindex = 0
        for i in range(Nx):
            for j in range(Ny):
                for k in range(Nz):
                    V[vindex] = params[0]*X[i] ** 2 + params[1]*Y[j] ** 2 + params[2]*Z[k] ** 2
                    vindex = vindex + 1
        return V
     
    # Only eigenvalues will be returned!
    evl = schrodinger3D(xmin, xmax, Nx, ymin, ymax, Ny, zmin, zmax, Nz, Vfun, params, neigs, E0, False)
     
    indices = np.argsort(evl)
    print("Energy eigenvalues:")
    for i,j in enumerate(evl[indices]):
        print("{}: {:.2f}".format(i + 1, np.real(j)))
         
    return sorted(evl)

In [3]:
print(sho_eigenenergies())